In [29]:
import json
import arrow
from building_depot import DataService
import pandas as pd
from collections import defaultdict

In [20]:
with open("config/ion_config.json", "r") as fp:
    configs = json.load(fp)
    base_url = configs['base_url']
    start_time = arrow.get(configs['start_time']).datetime
    end_time = arrow.get(configs['end_time']).datetime
with open("config/bd2_3secrets.json", "r") as fp:
    cred = json.load(fp)
    apikey = cred['api_key']
    username = cred['username']
    
datadir = "./data/"

In [32]:
def make_series(bd_data):
    time_list = list()
    value_list = list()
    for datum in data:
        time_list.append(list(datum.keys())[0])
        value_list.append(list(datum.values())[0])
    return pd.Series(index=time_list, data=value_list)

In [33]:
bd = DataService(base_url, apikey, username)
orig_metadata_df = pd.DataFrame.from_csv('metadata/ion.txt', )

In [34]:
srcid_dict = defaultdict(dict)
for srcid, row in orig_metadata_df.iterrows():
    uuid = row['uuid']
    data = bd.get_timeseries_datapoints(uuid, "PresentValue", start_time, end_time)['timeseries']
    if len(data)>1:
        sensor = bd.view_sensor(uuid)
        srcid_dict[srcid]['bd_uuid'] = uuid
        srcid_dict[srcid]['name'] = sensor['source_identifier']
        make_series(data).to_csv(datadir+srcid+'.csv')

In [35]:
with open('metadata/raw_ion_metadata.json', 'w') as fp:
    json.dump(srcid_dict, fp)